In [1]:
import os 
import folium   
from folium import plugins
import pandas as pd
import numpy as np
import datetime
    

In [2]:
# ## Data Prep

In [3]:
process=False
#process=True

if process:    
    data=pd.read_csv("export_european_capitals.csv")
    data.head()
    data.sort_values("ID_event").head()

    print(len(data))

    for line in data.itertuples():
        if data.iloc[line.Index]['dist']==0:
            data['dist'].at[line.Index]=7 #arbitrary for milan
    data.head()   

    data.sort_values(["ID_event","dist"], inplace=True)
    data

    ddrop=data.drop_duplicates(subset="ID_event" , keep="last")
    ddrop=ddrop.drop("ID_member", axis=1)
    #ddrop.sort_values("ID_event")
    ddrop=ddrop.reset_index()
    ddrop.head()

    d=data.groupby("ID_event")['dist'].quantile(0.75).reset_index()
    print(d.head())
    d.rename(columns={'dist':'maxdist'}, inplace=True)
    d2=data.groupby("ID_event")['dist'].mean().reset_index()
    d2.rename(columns={'dist':'meandist'}, inplace=True)
    ddrop=ddrop.merge(d, on="ID_event")
    ddrop=ddrop.merge(d2, on="ID_event")
    
    print(ddrop.head())

    d1=data.groupby("ID_event")['ID_member'].count().reset_index()
    print(d1.head())

    d1.rename(columns={'ID_member':'count_part'}, inplace=True)
    ddrop=ddrop.merge(d1, on="ID_event")
    del d1
    del d2
    del d
    del data
    ddrop.head()
    event_df=pd.read_csv("../../csv/struttura/event.csv")

    event_df['date']=str
    for line in event_df.itertuples():
        try:
            event_df['date'].at[line.Index]=str(datetime.datetime.fromtimestamp(int(float(line.time)/1000)).strftime('%c'))
        except:
            event_df['date'].at[line.Index]="Non disponibile"
    #event_df.head() 
    
    ddrop=ddrop.merge(event_df, left_on="ID_event", right_on="_key")
    #ddrop.head()
    del event_df
    topic_df=pd.read_csv("event_topic_keywords.csv")
    ddrop=ddrop.merge(topic_df,left_on="ID_event", right_on="ID").reset_index().drop("level_0",1)
    del topic_df
    ddrop.to_csv("post_proc.csv")
    
    

else:
    ddrop=pd.read_csv("post_proc.csv")
#ddrop1=pd.read_csv("post_proc.csv")
ddrop.head()
#topic_df.head()

,Unnamed: 0,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,...,count_part,_key,event_name,event_url,time,date,ID,City,Country,Keywords
0,0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000,...,1,106866582,Envie d\u00eatre form\u00e9 \u00e0 la M\u00e9d...,https://www.meetup.com/Lets-talk-about-relatio...,NONE,Non disponibile,106866582,Paris,fr,"['Support', 'Social', 'Education', 'Well-being..."
1,1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000,...,1,115197702,Carlinada en Madrid - Pug Playtime in Madrid,https://www.meetup.com/MadridPugClub/events/11...,NONE,Non disponibile,115197702,Madrid,es,['Pets']
2,2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219,...,1,123953142,Game Development Workshop Series - Session 05,https://www.meetup.com/London-Game-Developers/...,NONE,Non disponibile,123953142,London,gb,['Tech']
3,3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000,...,1,128105672,Dinner and Drinks Cruise @ IJ Amsterdam,https://www.meetup.com/Americans-in-Amsterdam/...,NONE,Non disponibile,128105672,Amsterdam,nl,"['Social', 'Singles', 'Food & Drink', 'Languag..."
4,4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657,...,1,131523832,Work in Fashion? Love Fashion?,https://www.meetup.com/Shop-Emerging-Fashion-D...,NONE,Non disponibile,131523832,London,gb,"['Social', 'Fashion', 'Photography', 'Business']"


In [4]:
#ddrop[ddrop.g_city=="Stockholm"].reset_index()
#print(len(ddrop))
#filtering participants
ddrop=ddrop[ddrop.count_part>1].reset_index()
ddrop.head()

,level_0,Unnamed: 0,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,...,count_part,_key,event_name,event_url,time,date,ID,City,Country,Keywords
0,12,12,52800,196982592,Madrid,40.42,-3.71,Peque\u00f1o Cine Estudio,40.432446,-3.705431,...,2,196982592,Sesi\u00f3n Continua,https://www.meetup.com/Madrid-Independent-Cine...,1575574200000,Thu Dec 5 20:30:00 2019,196982592,Madrid,es,"['Films', 'Languages']"
1,28,28,114230,230113887,London,51.52,-0.10,NaN,NaN,NaN,...,2,230113887,"Approaching Someone, Dating and Relationship T...",https://www.meetup.com/connecting-muslims/even...,1612116000000,Sun Jan 31 19:00:00 2021,230113887,London,gb,"['Beliefs', 'Singles']"
2,29,29,114232,230114341,London,51.52,-0.10,NaN,NaN,NaN,...,2,230114341,"Health, Well being and Exercise Tips (ONLINE ...",https://www.meetup.com/connecting-muslims/even...,1612202400000,Mon Feb 1 19:00:00 2021,230114341,London,gb,"['Beliefs', 'Singles']"
3,42,42,7650,235237185,London,51.52,-0.10,NaN,NaN,NaN,...,15,235237185,Whats App Group for meetups,https://www.meetup.com/London-Fun-meetup-group...,1559345400000,Sat Jun 1 01:30:00 2019,235237185,London,gb,"['Outdoors', 'Social', 'Singles', 'Films', 'Fo..."
4,46,46,153312,237782182,Paris,48.86,2.34,NaN,NaN,NaN,...,5,237782182,DDD et Microservices sont sur un bateau,https://www.meetup.com/DDD-Paris/events/237782...,NONE,Non disponibile,237782182,Paris,fr,['Tech']


## Let's build a map

In [28]:
#legend template
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Bacino d'utenza</li>
    <!-- questo è un commento <li><span style='background:orange;opacity:0.7;'></span>Medium</li>
    <li><span style='background:green;opacity:0.7;'></span>Small</li> -->

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""


html = """
    <h3> Event ID  : </h3><p>{}</p>
    <h3> Event Name: </h3><p></p>
    <h3> Location: </h3><p>{}</p>
    """
html1 = """
    <h2> Welcome to the meetup event map!  </h2>
    <h3> Some tips: </h3><p>{}</p>
    <h3> Some other tips: </h3><p>{}</p>
    """
html2 = """
<div style="
    position: fixed; 
    top: 80px;
    left: 80px;
    width: 300px;
    height: 340px;
    z-index:9999;
    font-size:14px;
    ">
    <h4> ID Evento: <a style="color:#e6194b;font-size:70%;">{}</a></h4>
    <h4> Event Data: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h4> Location: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h4> {} dell'evento: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h5> Raggio del bacino d'utenza: <a style="color:#e6194b;font-size:95%;">{} Km</a> </h5>
    <h4> Partecipanti: <a style="color:#e6194b;font-size:80%;">{}</a></h4>
    
</div>
<div style="
    position: fixed; 
    top: 70px;
    left: 70px;
    width: 320px;
    height: 360px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.7;
    ">
</div>
    """    

legend_html = """
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; Cool Legend <br>
     &nbsp; East &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:green”></i><br>
     &nbsp; West &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:red”></i>
      </div>
     """
legend_html2 = '''
<div style="
    position: fixed; 
    bottom: 50px;
    left: 80px;
    width: 250px;
    height: 40px;
    z-index:9999;
    font-size:14px;
    ">
    <p><a style="color:#e6194b;font-size:150%;margin-left:20px;">⭕</a>&emsp;Bacino d'utenza </p>
</div>
<div style="
    position: fixed; 
    bottom: 50px;
    left: 80px;
    width: 150px;
    height: 40px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.9;
    ">
</div>
'''

#html_file = open('widget.html','r')
#html_file=html_file.read()
#html_file

In [29]:
#ddrop=ddrop.sort_values('maxdist', ascending=False).reset_index()
#ddrop.head()

In [30]:
n=5000
ddrop.maxdist=ddrop.maxdist.mask(ddrop.maxdist>n,n)
ddrop.head()

,level_0,Unnamed: 0,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,...,count_part,_key,event_name,event_url,time,date,ID,City,Country,Keywords
0,12,12,52800,196982592,Madrid,40.42,-3.71,Peque\u00f1o Cine Estudio,40.432446,-3.705431,...,2,196982592,Sesi\u00f3n Continua,https://www.meetup.com/Madrid-Independent-Cine...,1575574200000,Thu Dec 5 20:30:00 2019,196982592,Madrid,es,"['Films', 'Languages']"
1,28,28,114230,230113887,London,51.52,-0.10,NaN,NaN,NaN,...,2,230113887,"Approaching Someone, Dating and Relationship T...",https://www.meetup.com/connecting-muslims/even...,1612116000000,Sun Jan 31 19:00:00 2021,230113887,London,gb,"['Beliefs', 'Singles']"
2,29,29,114232,230114341,London,51.52,-0.10,NaN,NaN,NaN,...,2,230114341,"Health, Well being and Exercise Tips (ONLINE ...",https://www.meetup.com/connecting-muslims/even...,1612202400000,Mon Feb 1 19:00:00 2021,230114341,London,gb,"['Beliefs', 'Singles']"
3,42,42,7650,235237185,London,51.52,-0.10,NaN,NaN,NaN,...,15,235237185,Whats App Group for meetups,https://www.meetup.com/London-Fun-meetup-group...,1559345400000,Sat Jun 1 01:30:00 2019,235237185,London,gb,"['Outdoors', 'Social', 'Singles', 'Films', 'Fo..."
4,46,46,153312,237782182,Paris,48.86,2.34,NaN,NaN,NaN,...,5,237782182,DDD et Microservices sont sur un bateau,https://www.meetup.com/DDD-Paris/events/237782...,NONE,Non disponibile,237782182,Paris,fr,['Tech']


In [31]:
def zoom_on_radius(radius):
    if radius >=10000:
        return 2
    if (radius >=1000 and radius <10000):
        return 3
    if (radius >= 500 and radius<1000):
        return 5
    if (radius >= 100 and radius<5000):
        return 6
    if (radius >= 50 and radius<100):
        return 8
    else:
        return 10

In [32]:
import branca #brrr
from folium.plugins import MarkerCluster
from folium.features import DivIcon, FeatureGroup
from folium import IFrame

import base64 #jpeg encoding


stop=True
n_stop=100

capital_list=["Roma", "Stockholm",  "London", ""]
coord_list=[]
#for capital in capital_list:
#    coord=ddrop1[ddrop1.g_city==capital].reset_index()

coord_list.append([41.9, 12.48])#Roma
coord_list.append([59.33, 18.07])#stockholm
coord_list.append([51.50, -0.12])#Londra
#coord_list.append([,])
print(coord_list)


m = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5, min_zoom=2)
mc=MarkerCluster()

radius_events = FeatureGroup(name='Events')
heat_events= FeatureGroup(name='Country')

country_list=["italy","sweden", "gb", "finland", "france", "espana", "portugal", "germany", "netherlands"]

dict_country={"italy":"Italy","sweden":"Sweden", "gb":"Great Britain", "finland":"Finlad", "france":"France", "espana":"Spain", "portugal":"Portugal", "germany":"Germany", "netherlands":"Netherlands"}
#coord_list=
for i  in range(len(country_list)):
    
    encoded = base64.b64encode(open("../heatmap_"+country_list[i]+".jpg", 'rb').read())#.decode()                           
    html4 = '<center><img src="data:image/jpg;base64,{}"><center>'.format
    #print(20*'-',encoded.decode('UTF-8'))
    f = branca.element.Figure()
    iframe = IFrame(html4(encoded.decode('UTF-8')), width=1200, height=600)
    h_popup = folium.Popup(iframe, max_width=2650)
    heat_events.add_child(
            folium.Marker(
            location = coord_list[i], 
            popup=h_popup,
            )
        ).add_to(m)
    country=dict_country[country_list[i]]
    f.html.add_child(branca.element.Element('<center><h1>{}<a style="color:#e6194b;font-size:100%;">{}</a></h1></center>'.format("Country: ",country)))
    f.add_to(iframe)
    #f.add_to_(m)
    #m.add_to(f)
    if i==2:
        break

for line in ddrop.itertuples():
    
        if str(line.v_lat) != "nan" : #where exists use venue lat/lon
            lat=line.v_lat
        else: #elsewhere use group lat
            lat=line.g_lat
            #print("g_lat: ",lat)
        if str(line.v_lon) != "nan": #where exists use venue lat/lon
            lon=line.v_lon
        else: #elsewhere use group lat
            lon=line.g_lon

        f = branca.element.Figure()
        
        m1=folium.Map([float(lat), float(lon)], zoom_start=zoom_on_radius(line.maxdist), min_zoom=2) #eventualmente aggiungere il livello di zoom in base al raggio può essere fyco
        Topic=""
        for topic in line.Keywords.split():
            topic=topic.replace("[","").replace("]","").replace("'","")
            Topic+=topic
        if len(Topic)==1:
            T="Topic"
        else:
            T="Topics"
        Topic=str(Topic)
        '''
        iframe = branca.element.IFrame(
            html=html.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], Topic), 
            width=400, height=240
        )
        
        popup1 = folium.Popup(iframe, show=True)
        '''
        f.html.add_child(branca.element.Element('<center><h3><a href="{}" target="_blank" style="color:#e6194b;font-size:100%;">{}</a></h3></center>'.format(line.event_url,line.event_name)))
        #f.html.add_child(branca.element.Element(html_file))
        if line.maxdist<line.meandist:
            meandist=line.maxdist
            maxdist=line.meandist
        else:
            meandist=line.meandist
            maxdist=line.maxdist
            
        m1.add_child(folium.Circle(
          location=[float(lat), float(lon)],
          #popup=popup1,
          #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
          radius=int(maxdist)*1000, #km->m
          color='crimson',
          fill=False,
          fill_color='crimson',
          fill_opacity=0.1,
          weight=0.5
        ))

        m1.add_child(folium.Circle(
          location=[float(lat), float(lon)],
          #popup=popup1,
          #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
          radius=int(meandist)*1000, #km->m
          color='crimson',
          fill=True,
          fill_color='crimson',
          fill_opacity=0.2,
          weight=0.1
        ))
       
            

        f.html.add_child(branca.element.Element(legend_html2))
        
        #html_now=html2.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], line.count_part)
        html_now=html2.format(line.ID_event, line.date, line.venue_name, T, Topic, int(meandist), line.count_part)
        
        f.html.add_child(branca.element.Element(html_now))
        
        #f.add_child(legend)
        
        #f.html.add_child(branca.element.Element("<h1>Bacino d'utenza</h1>"))
        
        m1.add_to(f)

        iframe = branca.element.IFrame(width=1000, height=600)
        f.add_to(iframe)

        # Let's put the IFrame in a Popup
        popup = folium.Popup(iframe, max_width=2650)
        
        mc.add_child(
            folium.Marker(
            location = [float(lat), float(lon)], 
            popup=popup,
            ))
        
        if stop:           
            if line.Index is n_stop:
                break
                
radius_events.add_child(mc)        
m.add_child(radius_events)
plugins.Fullscreen(
    
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

#macro = MacroElement()

#macro._template = Template(template)

#m.get_root().add_child(macro)
#map.get_root().add_child(macro1)

folium.LayerControl().add_to(m)
    
#Save it as html
m.save('europe_capitals.html')
print("finished")
m

[[41.9, 12.48], [59.33, 18.07], [51.5, -0.12]]
finished


In [57]:
import branca #brrr
from folium.plugins import MarkerCluster
from folium.features import DivIcon, FeatureGroup
from folium import IFrame

import base64 #jpeg encoding


#stop=True
n_stop=200

capital_list=["Roma", "Stockholm",  "London"]
coord_list=[]
#for capital in capital_list:
#    coord=ddrop1[ddrop1.g_city==capital].reset_index()

coord_list.append([41.9, 12.48])#Roma
coord_list.append([59.33, 18.07])#stockholm
coord_list.append([51.50, -0.12])#Londra
#coord_list.append([,])
print(coord_list)

from ipywidgets import interact

stop_l=[100,200,300,400,500]
stop=[True, False]

@interact(city=capital_list, stop=stop,n_stop=stop_l)
def show_canned_examples(city,stop,n_stop):
    
    m = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5, min_zoom=2)
    mc=MarkerCluster()

    radius_events = FeatureGroup(name='Events')
    heat_events= FeatureGroup(name='Country')

    country_list=["italy","sweden", "gb", "finland", "france", "espana", "portugal", "germany", "netherlands"]

    dict_country={"italy":"Italy","sweden":"Sweden", "gb":"Great Britain", "finland":"Finlad", "france":"France", "espana":"Spain", "portugal":"Portugal", "germany":"Germany", "netherlands":"Netherlands"}
    #coord_list=
    for i  in range(len(country_list)):

        encoded = base64.b64encode(open("../heatmap_"+country_list[i]+".jpg", 'rb').read())#.decode()                           
        html4 = '<center><img src="data:image/jpg;base64,{}",width="70%" height="70%"><center>'.format
        #print(20*'-',encoded.decode('UTF-8'))
        f = branca.element.Figure()
        iframe = IFrame(html4(encoded.decode('UTF-8')), width=800, height=600)
        h_popup = folium.Popup(iframe, max_width=800)
        heat_events.add_child(
                folium.Marker(
                location = coord_list[i], 
                popup=h_popup,
                )
            ).add_to(m)
        country=dict_country[country_list[i]]
        f.html.add_child(branca.element.Element('<center><h1>{}<a style="color:#e6194b;font-size:100%;">{}</a></h1></center>'.format("Country: ",country)))
        f.add_to(iframe)
        #f.add_to_(m)
        #m.add_to(f)
        if i==2:
            break

    for count,line in enumerate(ddrop.itertuples()):
        if line.g_city == city:
            if str(line.v_lat) != "nan" : #where exists use venue lat/lon
                lat=line.v_lat
            else: #elsewhere use group lat
                lat=line.g_lat
                #print("g_lat: ",lat)
            if str(line.v_lon) != "nan": #where exists use venue lat/lon
                lon=line.v_lon
            else: #elsewhere use group lat
                lon=line.g_lon
            
            if line.maxdist<line.meandist:
                meandist=line.maxdist
                maxdist=line.meandist
            else:
                meandist=line.meandist
                maxdist=line.maxdist

            f = branca.element.Figure()

            m1=folium.Map([float(lat), float(lon)], zoom_start=zoom_on_radius(maxdist), min_zoom=2) #eventualmente aggiungere il livello di zoom in base al raggio può essere fyco
            Topic=""
            for topic in line.Keywords.split():
                topic=topic.replace("[","").replace("]","").replace("'","")
                Topic+=topic
            if len(Topic)==1:
                T="Topic"
            else:
                T="Topics"
            Topic=str(Topic)
            '''
            iframe = branca.element.IFrame(
                html=html.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], Topic), 
                width=400, height=240
            )

            popup1 = folium.Popup(iframe, show=True)
            '''
            f.html.add_child(branca.element.Element('<center><h3><a href="{}" target="_blank" style="color:#e6194b;font-size:100%;">{}</a></h3></center>'.format(line.event_url,line.event_name)))
            #f.html.add_child(branca.element.Element(html_file))
            
            m1.add_child(folium.Circle(
              location=[float(lat), float(lon)],
              #popup=popup1,
              #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
              radius=int(maxdist)*1000, #km->m
              color='crimson',
              fill=False,
              fill_color='crimson',
              fill_opacity=0.1,
              weight=0.5
            ))

            m1.add_child(folium.Circle(
              location=[float(lat), float(lon)],
              #popup=popup1,
              #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
              radius=int(meandist)*1000, #km->m
              color='crimson',
              fill=True,
              fill_color='crimson',
              fill_opacity=0.2,
              weight=0.1
            ))



            f.html.add_child(branca.element.Element(legend_html2))

            #html_now=html2.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], line.count_part)
            html_now=html2.format(line.ID_event, line.date, line.venue_name, T, Topic, int(meandist), line.count_part)

            f.html.add_child(branca.element.Element(html_now))

            #f.add_child(legend)

            #f.html.add_child(branca.element.Element("<h1>Bacino d'utenza</h1>"))

            m1.add_to(f)

            iframe = branca.element.IFrame(width=1000, height=600)
            f.add_to(iframe)

            # Let's put the IFrame in a Popup
            popup = folium.Popup(iframe, max_width=2650)

            mc.add_child(
                folium.Marker(
                location = [float(lat), float(lon)], 
                popup=popup,
                ))
           
            if count == n_stop:
                break

    radius_events.add_child(mc)        
    m.add_child(radius_events)
    plugins.Fullscreen(

        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(m)

    #macro = MacroElement()

    #macro._template = Template(template)

    #m.get_root().add_child(macro)
    #map.get_root().add_child(macro1)

    folium.LayerControl().add_to(m)

    #Save it as html
    m.save('try.html')
    print("finished")
    return m



[[41.9, 12.48], [59.33, 18.07], [51.5, -0.12]]


interactive(children=(Dropdown(description='city', options=('Roma', 'Stockholm', 'London'), value='Roma'), Dro…

In [10]:
#Heatmap

In [77]:
go=True
if go:
    topic_list=[]
    for line in ddrop.itertuples():
        #keywords=line.Keywords.split(",")
        #print(keywords)
        
        for topic in line.Keywords.split(","):
            topic=topic.replace("[","").replace("]","").replace("'","").replace(",","")
            Topic.append(topic)
        for topic in Topic:
            if  topic not in topic_list:
                topic_list.append(topic)
        
    for elem in topic_list:
        print(elem)
        

KeyboardInterrupt: 

In [66]:
@interact(topic=topic_list)
def show_canned_examples(topic):
    lat_df=ddrop[['v_lat', 'v_lon']]
    print(len(ddrop))
    str(lat_df.v_lat[0]) == "nan"
    
    for line in lat_df.itertuples():
        if str(line.v_lat) == "nan":
            lat_df['v_lat'].at[line.Index]= ddrop['g_lat'].at[line.Index]
        if str(line.v_lon) == "nan":
            lat_df['v_lon'].at[line.Index]= ddrop['g_lon'].at[line.Index]
    len(lat_df)

    lat_df.isna().any()

    lat_df=lat_df.astype(float)

    from folium.plugins import HeatMap


    map_hooray = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5)


    # List comprehension to make out list of lists
    #heat_data = [[row['v_lat'],row['v_lon']] for index, row in lat_df.iterrows()]
    heat_data=[]
    for line in lat_df.itertuples():
        for topics in line.Keywords.split():
            topics=topics.replace("[","").replace("]","").replace("'","")
            Topic.append(topics)
        if topic in Topics:
            heat_data.append([line['v_lat'],line['v_lon']])
    # Plot it on the map
    HeatMap(heat_data).add_to(map_hooray)

    # Display the map
    map_hooray.save("/home/dario/NeoMeetup/Visualizations/map/heat_try.html")
    display(map_hooray)

interactive(children=(Dropdown(description='topic', options=('Films,', 'Languages', 'Beliefs,', 'Singles', 'Ou…